In [1]:
import findspark

In [2]:
findspark.init('/home/joe/spark-2.4.3-bin-hadoop2.7')

In [3]:
import pyspark

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.master("local").appName('health').getOrCreate()

In [6]:
df = spark.read.json('data/healthcare06')

In [7]:
df.printSchema()

root
 |-- _corrupt_record: string (nullable = true)
 |-- contributors: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- display_url: string (nullable = true)
 |    |    |    |-- expanded_url: string (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- id_

In [8]:
df.head()

Row(_corrupt_record=None, contributors=None, coordinates=None, created_at='Mon Oct 02 18:37:55 +0000 2017', display_text_range=[0, 90], entities=Row(hashtags=[], media=None, symbols=[], urls=[Row(display_url='twitter.com/goodoldcatchy/…', expanded_url='https://twitter.com/goodoldcatchy/status/914832764179931136', indices=[67, 90], url='https://t.co/gbuIXjIjJC')], user_mentions=[Row(id=23141473, id_str='23141473', indices=[3, 17], name='Calvin', screen_name='calvinstowell')]), extended_entities=None, favorite_count=0, favorited=False, full_text="RT @calvinstowell: I'm American, and I don't understand it either. https://t.co/gbuIXjIjJC", geo=None, id=914922409613856800, id_str='914922409613856768', in_reply_to_screen_name=None, in_reply_to_status_id=None, in_reply_to_status_id_str=None, in_reply_to_user_id=None, in_reply_to_user_id_str=None, is_quote_status=True, lang='en', place=None, possibly_sensitive=False, quoted_status=None, quoted_status_id=914832764179931100, quoted_status_id_str

In [9]:
df.createOrReplaceTempView("health")

In [10]:
df1 = spark.sql("SELECT DISTINCT created_at AS date,user.id AS user_id,user.name AS user_name,favorite_count,retweet_count,full_text AS text,user.location AS user_location FROM health WHERE lang = 'en' AND user.location REGEXP 'Newfoundland and Labrador|, NL$|Alberta|, AB$|Saskatchewan|, SK$|Prince Edward Island|, PE$|British Columbia|, BC$|Manitoba|, MB$|New Brunswick|, NB$|Nova Scotia|, NS$|Quebec|, QC$|Ontario|, ON$'")

In [11]:
df1.count()

63688

In [12]:
df1.head()

Row(date='Sun Jul 30 07:45:31 +0000 2017', user_id=380664031, user_name='James Toupin', favorite_count=0, retweet_count=7845, text="RT @TheRickyDavila: MSNBC's @AliVelshi calls out EVERY Senator who were paid off by the insurance companies. This is disgusting. SHARE THIS…", user_location='Edmonton, Alberta, Canada')

In [13]:
df1.show()

+--------------------+------------------+--------------------+--------------+-------------+--------------------+--------------------+
|                date|           user_id|           user_name|favorite_count|retweet_count|                text|       user_location|
+--------------------+------------------+--------------------+--------------+-------------+--------------------+--------------------+
|Sun Jul 30 07:45:...|         380664031|        James Toupin|             0|         7845|RT @TheRickyDavil...|Edmonton, Alberta...|
|Sun Jul 30 14:31:...|         293774123|    Jan Ravensbergen|             0|          600|RT @mattmfm: A we...|Montreal, Quebec,...|
|Sun Jul 30 14:55:...|         127309627|           Paul Rose|             0|            0|Riddle me this - ...|    Ontario - Canada|
|Sun Jul 30 15:19:...|         126498319|   Peggy S. Mitchell|             0|          429|RT @ThePlumLineGS...|Edmonton, Alberta...|
|Sun Jul 30 16:02:...|789512359693938700|         Mike Almond|

In [12]:
df1.coalesce(1).write.mode("overwrite").csv('part06')